In [1]:
import pathlib, os, sys

In [2]:
# Update PythonPath to resolve imports from src/
src_dir = str(pathlib.Path(os.getcwd()).parent.parent.absolute())
sys.path.append(src_dir)
src_dir

'/Users/divi/Desktop/RosettaCV/src'

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
from models.CMC.contrastive_multiview import *
from models.CMC.ResNetEncoder import ResNetEncoder
from data_loader.MultiviewDatasets import *
from utils.util import split_dataset
from losses.cmc_losses import contrastive_loss
from models.CMC.contrastive_multiview import CMCTrainer, View, WrapperModel
from models.CMC.language_models import TextEncoder

In [5]:
path2data = "../../data/train2014"
path2json = "../../data/annotations/captions_train2014.json"
train_loader, val_loader = get_coco_data_loaders(path2data, path2json, 32, 0.7)

loading annotations into memory...
Done (t=0.61s)
creating index...
index created!


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fe1 = ResNetEncoder(device, latent_dim=512)
lm1 = TextEncoder(latent_dim=512)

In [7]:
view1 = View(fe1)
textview = View(lm1)
model = WrapperModel([view1, textview], 512)

In [8]:
sample = next(iter(train_loader))

In [9]:
sample[1]

('A woman picking up a cup cake from a booth.\n',
 'Planes are outside on the tarmac waiting to be prepared for flight.',
 'A man wearing a hat carrying two lamps in a field.',
 'a statnue of a bird on a branch next to a tall building ',
 'a funny shaped toilet with a dark brown lid',
 'Black truck with a RV trailer attached to the hitch.',
 'A brown horse standing next to an animal laying on the ground.',
 'A car moving fast on the intersection at night.  ',
 'A freshly baked pie with a browned crust',
 'A very big cute polar bear laying in snow.',
 'There is an empty bathroom, with vanity, toilet, and wall mirror.',
 'A couple of men riding horses on top of a green field.',
 'A little bird sleeps in a garden planter.',
 'A person on a skateboard does a trick in a bowl.',
 'A suitcase and a cat on the ground.',
 'A couple of boats floating along a river.',
 'A room with a blue chair green chair and wooden chair',
 'A closed umbrella and beach chairs on a beach',
 'Six jets are flying 

In [10]:
trainer = CMCTrainer(model, contrastive_loss, train_loader, val_loader, num_decodings_per_step=0)
trainer.train(500, save_to='baseline.p')

wandb: Currently logged in as: cal-launchpad (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.28 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


KeyboardInterrupt: 